# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!cp '/content/gdrive/My Drive/EVA/stanfordSentimentTreebank.zip' stanfordSentimentTreebank.zip
!unzip -q -o stanfordSentimentTreebank.zip -d stanfordSentimentTreebank

In [3]:
import pandas as pd
df = pd.read_csv('/content/stanfordSentimentTreebank/stanfordSentimentTreebank/datasetSentences.txt',sep='\t')
df.tail()

,sentence_index,sentence
11850,11851,A real snooze .
11851,11852,No surprises .
11852,11853,We 've seen the hippie-turned-yuppie plot befo...
11853,11854,Her fans walked out muttering words like `` ho...
11854,11855,In this case zero .


In [4]:
df.shape

(11855, 2)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11855 entries, 0 to 11854
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   sentence_index  11855 non-null  int64 
 1   sentence        11855 non-null  object
dtypes: int64(1), object(1)
memory usage: 185.4+ KB


## Defining Fields

In [6]:
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [7]:
class StanfordDatasetReader():
  def __init__(self, sst_dir, split_idx):
    
    merged_dataset = self.get_merged_dataset(sst_dir)
    merged_dataset['sentiment values'] = merged_dataset['sentiment values'].astype(float)
    self.dataset = merged_dataset[merged_dataset["splitset_label"] == split_idx]
    # self.dataset["Revised_Sentiment"] = self.discretize_label(self.dataset.iloc[5])
    self.dataset['Revised_sentiment values'] = self.dataset.apply(lambda x: labelfunc(x["sentiment values"]), axis=1)
    # train_st_data['Revised_sentiment values'] = train_st_data.apply(lambda x: myfunc(x["sentiment values"]), axis=1)
  # https://github.com/iamsimha/conv-sentiment-analysis/blob/master/code/dataset_reader.py
  def get_merged_dataset(self, sst_dir):

    sentiment_labels = pd.read_csv(os.path.join(sst_dir, "sentiment_labels.txt"), sep="|")
    sentence_ids = pd.read_csv(os.path.join(sst_dir, "datasetSentences.txt"), sep="\t")
    dictionary = pd.read_csv(os.path.join(sst_dir, "dictionary.txt"), sep="|", names=['phrase', 'phrase ids'])
    train_test_split = pd.read_csv(os.path.join(sst_dir, "datasetSplit.txt"))
    sentence_phrase_merge = pd.merge(sentence_ids, dictionary, left_on='sentence', right_on='phrase')
    sentence_phrase_split = pd.merge(sentence_phrase_merge, train_test_split, on='sentence_index')
    return pd.merge(sentence_phrase_split, sentiment_labels, on='phrase ids').sample(frac=1)

  def discretize_label(self, label):
    print(type(label))
    if label <= 0.2: return 0
    if label <= 0.4: return 1
    if label <= 0.6: return 2
    if label <= 0.8: return 3
    return 4

  def word_to_index(self, word):
    if word in self.w2i:
      return self.w2i[word]
    else:
      return self.w2i["<OOV>"]

  def __len__(self):
    return self.dataset.shape[0]
    
  # def __getitem__(self, idx):
  #   return {"sentence": [self.word_to_index(x) for x in self.dataset.iloc[idx, 1].split()],
  #           "label": self.discretize_label(self.dataset.iloc[idx, 5])}
  def labelfunc(label):
    if label <= 0.5: return 0
    # if label <= 0.4: return 1
    # if label <= 0.6: return 2
    # if label <= 0.8: return 3
    return 1

  def get_data(self):
    return self.dataset

  def __getitem__(self, idx):
    return {"sentence": [x for x in self.dataset.iloc[idx, 1].split()],
            "label": self.discretize_label(self.dataset.iloc[idx, 5])}

In [8]:
def labelfunc(label):
  if label <= 0.4: return 0
  if label <= 0.6: return 1
  # if label <= 0.6: return 2
  # if label <= 0.8: return 3
  return 2

In [9]:
import os
def load_data(sst_dir="/content/stanfordSentimentTreebank/stanfordSentimentTreebank/"):
  train_st_data_cl = StanfordDatasetReader(sst_dir, 1).get_data()
  # train_st_data_cl['Revised_sentiment values'] = train_st_data.apply(lambda x: labelfunc(x["sentiment values"]), axis=1)
  test_st_data_cl = StanfordDatasetReader(sst_dir, 2).get_data()
  # test_st_data_cl['Revised_sentiment values'] = test_st_data_cl.apply(lambda x: labelfunc(x["sentiment values"]), axis=1)
  validation_st_data_cl = StanfordDatasetReader(sst_dir, 3).get_data()
  # validation_st_data_cl['Revised_sentiment values'] = validation_st_data_cl.apply(lambda x: labelfunc(x["sentiment values"]), axis=1)
  return train_st_data_cl,test_st_data_cl,validation_st_data_cl

In [10]:
train_st_data,test_st_data,validation_st_data = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [11]:
train_st_data.head()

,sentence_index,sentence,phrase,phrase ids,splitset_label,sentiment values,Revised_sentiment values
10442,10962,"Of all the Halloween 's , this is the most vis...","Of all the Halloween 's , this is the most vis...",225274,1,0.13889,0
2041,2124,"By no means a great movie , but it is a refres...","By no means a great movie , but it is a refres...",44818,1,0.69444,2
2871,3000,"Triumph of Love is a very silly movie , but th...","Triumph of Love is a very silly movie , but th...",70482,1,0.47222,1
5626,5884,It 's Tommy 's job to clean the peep booths su...,It 's Tommy 's job to clean the peep booths su...,106802,1,0.26389,0
11010,11561,"The script is smart , not cloying .","The script is smart , not cloying .",188644,1,0.75000,2


In [12]:
test_st_data.tail()

,sentence_index,sentence,phrase,phrase ids,splitset_label,sentiment values,Revised_sentiment values
177,180,A well-made thriller with a certain level of i...,A well-made thriller with a certain level of i...,13824,2,0.833330,2
984,1017,Nicolas Philibert observes life inside a one-r...,Nicolas Philibert observes life inside a one-r...,26190,2,0.888890,2
8540,8939,A movie that tries to fuse the two ` woods ' b...,A movie that tries to fuse the two ` woods ' b...,143246,2,0.333330,0
8338,8730,would seem to have a lock on the title of ugli...,would seem to have a lock on the title of ugli...,178923,2,0.097222,0
43,44,The story loses its bite in a last-minute happ...,The story loses its bite in a last-minute happ...,226498,2,0.111110,0


In [13]:
validation_st_data.head()

,sentence_index,sentence,phrase,phrase ids,splitset_label,sentiment values,Revised_sentiment values
7025,7352,"Less dizzying than just dizzy , the jaunt is p...","Less dizzying than just dizzy , the jaunt is p...",147127,3,0.25000,0
1165,1208,It 's a much more emotional journey than what ...,It 's a much more emotional journey than what ...,25732,3,0.75000,2
1445,1506,"If the first Men in Black was money , the seco...","If the first Men in Black was money , the seco...",223928,3,0.16667,0
1178,1223,So refreshingly incisive is Grant that for the...,So refreshingly incisive is Grant that for the...,26608,3,0.66667,2
7541,7897,"Some of their jokes work , but most fail miser...","Some of their jokes work , but most fail miser...",148746,3,0.20833,0


### Further NLP Augemnattion

In [14]:
!pip install nlpaug

     |████████████████████████████████| 389kB 10.0MB/s 


In [15]:
# !pip install transformers

In [16]:
## Lets do the NLP data augmentation
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

##### Some basic examples for understanding and then further data augmentation by these
- Substitute word by WordNet's synonym
- Swap word randomly
- Delete a set of contunous word will be removed randomly
- Delete word randomly augemnattion

In [17]:
import os
os.environ["model_dir"] = '../model'

In [18]:
model_dir = "/content/"

In [19]:
# validation_st_data.head()
train_st_data['sentence'].iloc[0]

"Of all the Halloween 's , this is the most visually unappealing ."

In [20]:
aug = naw.SynonymAug(aug_src='wordnet') ## Substitute word by WordNet's synonym¶

augmented_text = aug.augment(train_st_data['sentence'].iloc[0])
print("Original:")
print(train_st_data['sentence'].iloc[0])
print("Augmented Text:")
print(augmented_text)
train_st_data_SynonymAug_aug = train_st_data
train_st_data_SynonymAug_aug['sentence_aug'] = train_st_data_SynonymAug_aug.apply(lambda x: aug.augment(x['sentence']),axis=1)  ## Swap word randomly¶

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Original:
Of all the Halloween 's , this is the most visually unappealing .
Augmented Text:
Of all the Halloween ' s, this represent the to the highest degree visually unsympathetic.


In [21]:
aug = naw.RandomWordAug(action="swap") # Swap word randomly¶

augmented_text = aug.augment(train_st_data['sentence'].iloc[0])
print("Original:")
print(train_st_data['sentence'].iloc[0])
print("Augmented Text:")
print(augmented_text)
train_st_data_swap_aug = train_st_data
train_st_data_swap_aug['sentence_aug'] = train_st_data_swap_aug.apply(lambda x: aug.augment(x['sentence']),axis=1)  ## Swap word randomly¶

Original:
Of all the Halloween 's , this is the most visually unappealing .
Augmented Text:
Of all Halloween the s ', is this most the visually unappealing.


In [22]:
# aug = naw.RandomWordAug(action='crop',aug_p=0.5, aug_min=0)
# augmented_text = aug.augment(train_st_data['sentence'].iloc[0])  ## Delete a set of contunous word will be removed randomly¶

# print("Original:")
# print(train_st_data['sentence'].iloc[0])
# print("Augmented Text:")
# print(augmented_text)
# train_st_data_crop_aug = train_st_data
# train_st_data_crop_aug['sentence_aug'] = train_st_data_crop_aug.apply(lambda x: aug.augment(x['sentence']),axis=1)  ## Delete a set of contunous word will be removed randomly¶

In [24]:
text = 'The quick brown fox jumps over the lazy dog .'
# Augmenter that apply random word operation to textual input.Augmenter that apply randomly behavior for augmentation.
aug = naw.RandomWordAug()
augmented_data = aug.augment(text)
augmented_data

train_st_data_delete_aug = train_st_data
# train_st_data_aug[sentence_aug] = aug.augment(train_st_data_aug.loc["sentence"] )
#--Using position to slice Email using a lambda function
train_st_data_delete_aug['sentence_aug'] = train_st_data_delete_aug.apply(lambda x: aug.augment(x['sentence']),axis=1)  ## Delete word randomly augemnattion


In [25]:
print("Original:")
print(train_st_data_delete_aug['sentence'].iloc[0])
print("Augmented Text:")
print(train_st_data_delete_aug['sentence_aug'].iloc[0])

Original:
Of all the Halloween 's , this is the most visually unappealing .
Augmented Text:
Of all the Halloween ' s, the most.


In [27]:
train_st_data_delete_aug.head()

,sentence_index,sentence,phrase,phrase ids,splitset_label,sentiment values,Revised_sentiment values,sentence_aug
10442,10962,"Of all the Halloween 's , this is the most vis...","Of all the Halloween 's , this is the most vis...",225274,1,0.13889,0,"Of all the Halloween ' s, the most."
2041,2124,"By no means a great movie , but it is a refres...","By no means a great movie , but it is a refres...",44818,1,0.69444,2,"By no a great, it is a refreshingly forthright."
2871,3000,"Triumph of Love is a very silly movie , but th...","Triumph of Love is a very silly movie , but th...",70482,1,0.47222,1,"Triumph of Love is very silly, the silliness a..."
5626,5884,It 's Tommy 's job to clean the peep booths su...,It 's Tommy 's job to clean the peep booths su...,106802,1,0.26389,0,' s Tommy ' job clean peep booths surrounding ...
11010,11561,"The script is smart , not cloying .","The script is smart , not cloying .",188644,1,0.75000,2,"The script is, not."


In [36]:
## Now I need to add all these data frames
combined_data_aug = pd.concat([train_st_data_delete_aug, train_st_data_swap_aug, train_st_data_SynonymAug_aug], axis=0)
## after this, now I need to drop the sentence column and rename sentence_aug to sentence
combined_data_aug.drop('sentence', axis=1, inplace=True)
combined_data_aug.rename(columns = {'sentence_aug':'sentence'}, inplace = True) 

In [37]:
combined_data_aug.head()

,sentence_index,phrase,phrase ids,splitset_label,sentiment values,Revised_sentiment values,sentence
10442,10962,"Of all the Halloween 's , this is the most vis...",225274,1,0.13889,0,"Of all the Halloween ' s, the most."
2041,2124,"By no means a great movie , but it is a refres...",44818,1,0.69444,2,"By no a great, it is a refreshingly forthright."
2871,3000,"Triumph of Love is a very silly movie , but th...",70482,1,0.47222,1,"Triumph of Love is very silly, the silliness a..."
5626,5884,It 's Tommy 's job to clean the peep booths su...,106802,1,0.26389,0,' s Tommy ' job clean peep booths surrounding ...
11010,11561,"The script is smart , not cloying .",188644,1,0.75000,2,"The script is, not."


### Final Data Preparation

In [38]:
def get_final_data(train_st_data,test_st_data,validation_st_data,combined_data_aug):
  train_st_data_final = train_st_data.drop(['sentence_index','phrase','phrase ids','splitset_label','sentiment values'],axis=1)
  train_st_data_final.rename(columns = {'Revised_sentiment values': 'sentiment'}, inplace = True)

  combined_data_aug.drop(['sentence_index','phrase','phrase ids','splitset_label','sentiment values'],axis=1,inplace=True)
  combined_data_aug.rename(columns = {'Revised_sentiment values': 'sentiment'}, inplace = True)

  train_st_data_final_mixed = pd.concat([combined_data_aug, train_st_data_final], axis=0)

  train_st_data_final_mixed = train_st_data_final_mixed.reset_index(drop=True) ## This is being done because data.Example.fromlist was failing

  test_st_data_final = test_st_data.drop(['sentence_index','phrase','phrase ids','splitset_label','sentiment values'],axis=1)
  test_st_data_final.rename(columns = {'Revised_sentiment values': 'sentiment'}, inplace = True)
  test_st_data_final = test_st_data_final.reset_index(drop=True) ## This is being done because data.Example.fromlist was failing

  validation_st_data_final = validation_st_data.drop(['sentence_index','phrase','phrase ids','splitset_label','sentiment values'],axis=1)
  validation_st_data_final.rename(columns = {'Revised_sentiment values': 'sentiment'} , inplace = True)
  validation_st_data_final = validation_st_data_final.reset_index(drop=True) ## This is being done because data.Example.fromlist was failing

  return train_st_data_final_mixed, test_st_data_final, validation_st_data_final

In [39]:
train_st_data_final, test_st_data_final, validation_st_data_final = get_final_data(train_st_data,test_st_data,validation_st_data,combined_data_aug)

In [40]:
train_st_data_final.head()

,sentiment,sentence,sentence_aug
0,0,"Of all the Halloween ' s, the most.",NaN
1,2,"By no a great, it is a refreshingly forthright.",NaN
2,1,"Triumph of Love is very silly, the silliness a...",NaN
3,0,' s Tommy ' job clean peep booths surrounding ...,NaN
4,2,"The script is, not.",NaN


In [ ]:
# train_st_data_final.to_csv(r'train_st_data_final.csv', index = False)

In [41]:
train_st_data_final.shape

(32468, 3)

In [42]:
train_st_data_final.sentiment.value_counts()

2    13784
0    12488
1     6196
Name: sentiment, dtype: int64

In [43]:
validation_st_data_final.sentiment.value_counts()

2    417
0    408
1    219
Name: sentiment, dtype: int64

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [44]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [45]:
Sentence = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Sentiment = data.LabelField(tokenize ='spacy',is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [46]:
fields = [('sentence', Sentence),('sentiment',Sentiment)]

In [47]:
# saving the dataframe 
train_st_data_final.to_csv('train_st_data_final.csv', index=False) 
test_st_data_final.to_csv('test_st_data_final.csv', index=False) 
validation_st_data_final.to_csv('validation_st_data_final.csv', index=False) 

In [48]:
# df = pd.read_csv('train_st_data_final.csv')
# df.head()
# example_trng = [data.Example.fromlist([df.sentence[i],df.sentiment[i]], fields) for i in range(df.shape[0])] 

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [49]:
example_trng = [data.Example.fromlist([train_st_data_final.sentence[i],train_st_data_final.sentiment[i]], fields) for i in range(train_st_data_final.shape[0])] 
example_val = [data.Example.fromlist([validation_st_data_final.sentence[i],validation_st_data_final.sentiment[i]], fields) for i in range(validation_st_data_final.shape[0])] 

In [51]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

# twitterDataset = data.Dataset(example, fields)
train = data.Dataset(example_trng, fields)
valid = data.Dataset(example_val, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [ ]:
# (train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [52]:
(len(train), len(valid))

(32468, 1044)

An example from the dataset:

In [53]:
vars(train.examples[10])

{'sentence': ['Neil',
  'succeeded',
  'in',
  '.',
  '..',
  'the',
  'mystery',
  'back',
  'the',
  'springboard',
  'for',
  'a',
  'immediate',
  'mystery',
  'in',
  'the',
  'present',
  '.'],
 'sentiment': 2}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [54]:
MAX_VOCAB_SIZE = 25_000

Sentence.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

.vector_cache/glove.6B.zip: 862MB [06:28, 2.22MB/s]                           
 99%|█████████▉| 397883/400000 [00:16<00:00, 24067.24it/s]

In [55]:
Sentiment.build_vocab(train)

In [ ]:
# Sentence.build_vocab(train)
# Sentiment.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [56]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Sentiment.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Sentiment.vocab.stoi)

Size of input vocab :  17022
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 31962), (',', 26833), ('the', 17267), ('-', 13429), ("'", 13351), ('of', 12669), ('and', 12477), ('a', 12241), ('to', 8652), ('is', 7059)]
Labels :  defaultdict(<function _default_unk_index at 0x7f3932863c80>, {2: 0, 0: 1, 1: 2})


In [57]:
print('Size of input vocab : ', len(Sentence.vocab))
print('Size of label vocab : ', len(Sentiment.vocab))
print('Top 10 words appreared repeatedly :', list(Sentence.vocab.freqs.most_common(10)))
print('Labels : ', Sentiment.vocab.stoi)

Size of input vocab :  17022
Size of label vocab :  3
Top 10 words appreared repeatedly : [('.', 31962), (',', 26833), ('the', 17267), ('-', 13429), ("'", 13351), ('of', 12669), ('and', 12477), ('a', 12241), ('to', 8652), ('is', 7059)]
Labels :  defaultdict(<function _default_unk_index at 0x7f3932863c80>, {2: 0, 0: 1, 1: 2})


**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [59]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.sentence),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [61]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Sentence.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [62]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, bidirectional,pad_idx):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim,padding_idx = pad_idx)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           bidirectional = bidirectional,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        self.dropout = nn.Dropout(dropout)
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.dropout(self.fc(hidden))  
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
        # output = F.softmax(dense_outputs, dim=1)
            
        return output

In [63]:
# Define hyperparameters
size_of_vocab = len(Sentence.vocab)
embedding_dim = 100
num_hidden_nodes = 256
num_output_nodes = 3
num_layers = 3
dropout = 0.2
bidirectional = True
PAD_IDX = Sentence.vocab.stoi[Sentence.pad_token]

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout, bidirectional,PAD_IDX)

In [64]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17022, 100, padding_idx=1)
  (encoder): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=3, bias=True)
)
The model has 5,590,075 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [65]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-3)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    


In [66]:
pretrained_embeddings = Sentence.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([17022, 100])


In [67]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.0166, -0.4668,  2.0909,  ..., -1.4692,  0.4476, -0.7223],
        [-0.0791, -0.2089, -0.3442,  ...,  0.4657,  0.6297, -1.7395],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.2059,  0.8924,  0.4946,  ..., -0.0663, -0.2220, -0.1561],
        [ 0.2973, -0.1341,  0.6720,  ..., -0.3700, -0.3623,  0.7735],
        [-0.6763,  0.1671,  0.9469,  ..., -0.3254, -0.2345,  0.2310]])

In [68]:
UNK_IDX = Sentence.vocab.stoi[Sentence.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-0.2059,  0.8924,  0.4946,  ..., -0.0663, -0.2220, -0.1561],
        [ 0.2973, -0.1341,  0.6720,  ..., -0.3700, -0.3623,  0.7735],
        [-0.6763,  0.1671,  0.9469,  ..., -0.3254, -0.2345,  0.2310]])


In [69]:
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [70]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.sentence   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.sentiment)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.sentiment)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Evaluation Loop**

In [71]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.sentence
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.sentiment)
            acc = binary_accuracy(predictions, batch.sentiment)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [72]:
N_EPOCHS = 15
best_valid_loss = float('inf')
#freeze embeddings
model.embedding.weight.requires_grad = unfrozen = False

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 0.992 | Train Acc: 54.29%
	 Val. Loss: 0.933 |  Val. Acc: 60.83% 

	Train Loss: 0.919 | Train Acc: 62.74%
	 Val. Loss: 0.939 |  Val. Acc: 60.30% 

	Train Loss: 0.864 | Train Acc: 68.61%
	 Val. Loss: 0.927 |  Val. Acc: 61.78% 

	Train Loss: 0.825 | Train Acc: 72.79%
	 Val. Loss: 0.918 |  Val. Acc: 63.01% 

	Train Loss: 0.794 | Train Acc: 75.70%
	 Val. Loss: 0.925 |  Val. Acc: 61.12% 

	Train Loss: 0.766 | Train Acc: 78.52%
	 Val. Loss: 0.917 |  Val. Acc: 62.54% 

	Train Loss: 0.745 | Train Acc: 80.85%
	 Val. Loss: 0.918 |  Val. Acc: 62.35% 

	Train Loss: 0.731 | Train Acc: 82.18%
	 Val. Loss: 0.912 |  Val. Acc: 63.24% 

	Train Loss: 0.712 | Train Acc: 84.14%
	 Val. Loss: 0.934 |  Val. Acc: 60.85% 

	Train Loss: 0.702 | Train Acc: 85.24%
	 Val. Loss: 0.938 |  Val. Acc: 60.55% 

	Train Loss: 0.695 | Train Acc: 85.89%
	 Val. Loss: 0.933 |  Val. Acc: 61.12% 

	Train Loss: 0.692 | Train Acc: 86.22%
	 Val. Loss: 0.931 |  Val. Acc: 61.21% 

	Train Loss: 0.679 | Train Acc: 87.37%
	

## Model Testing

In [73]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [ ]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [ ]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [ ]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [ ]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [ ]:
!pip install googletrans==3.1.0a0

In [ ]:
import random
import googletrans
from googletrans import Translator

# import googletrans.Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)